
# **Diversificación Máxima**
En este cuaderno, exploramos la construcción de una cartera que **maximiza la diversificación**, utilizando la clase `MaximumDiversification` de `skfolio`.

La **diversificación máxima** se basa en la optimización del **ratio de diversificación**, definido como la relación entre la suma de las volatilidades ponderadas de los activos y la volatilidad total de la cartera. Este enfoque busca construir una cartera en la que los activos contribuyan de manera más equilibrada al riesgo total, reduciendo la concentración en pocos activos y mejorando la estabilidad del portafolio.


Antes de continuar, asegurémonos de que `skfolio` está instalado en el entorno de Python. Si no lo tienes instalado, ejecuta el siguiente comando:

In [ ]:
!pip install skfolio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.5/734.5 kB 7.1 MB/s eta 0:00:00


## **Carga y preparación de datos**
En esta sección, importamos las bibliotecas necesarias y cargamos un conjunto de datos compuesto por los precios diarios de 20 activos del índice S&P 500. Los datos abarcan el período del 2 de enero de 1990 al 28 de diciembre de 2022.

Para garantizar la validez de la estrategia y evitar la **fuga de datos**, se divide el conjunto de datos en subconjuntos de entrenamiento y prueba sin realizar mezclas aleatorias.


In [ ]:
# Importamos las bibliotecas necesarias
from plotly.io import show  # Para visualizar gráficos interactivos
from sklearn.model_selection import train_test_split  # Para dividir los datos en entrenamiento y prueba

# Importamos módulos de skfolio
from skfolio import Population  # Clase para manejar múltiples carteras
from skfolio.datasets import load_sp500_dataset  # Dataset con precios históricos de activos del S&P 500
from skfolio.optimization import EqualWeighted, MaximumDiversification  # Modelos de optimización de carteras
from skfolio.preprocessing import prices_to_returns  # Función para convertir precios en rendimientos

# Cargamos los precios históricos del S&P 500 (20 activos seleccionados)
prices = load_sp500_dataset()

# Convertimos los precios en rendimientos
X = prices_to_returns(prices)

# Dividimos los datos en un conjunto de entrenamiento (67%) y prueba (33%) sin mezclar los datos (shuffle=False)
X_train, X_test = train_test_split(X, test_size=0.33, shuffle=False)

## **Construcción del modelo de diversificación máxima**
En esta sección, creamos y ajustamos un modelo de optimización de cartera que **maximiza la diversificación**. Posteriormente, comparamos su desempeño con una cartera equiponderada para evaluar si la diversificación activa aporta beneficios en términos de riesgo y rendimiento.

**Creación del modelo de diversificación máxima**

El modelo de `MaximumDiversification` se entrena utilizando los datos históricos de rendimientos para encontrar una asignación óptima de pesos en los activos que maximice el **ratio de diversificación**.

Este enfoque busca distribuir el riesgo de manera más uniforme entre los activos, evitando concentraciones excesivas en pocos elementos del portafolio.

In [ ]:
# Creamos el modelo de diversificación máxima
model = MaximumDiversification()

# Entrenamos el modelo con el conjunto de datos de entrenamiento
model.fit(X_train)

# Obtenemos los pesos óptimos de cada activo en la cartera de diversificación máxima
model.weights_

array([8.33459971e-02, 6.74138299e-02, 2.93952123e-02, 8.57558650e-02,
       4.12145074e-02, 8.80360161e-09, 1.53457254e-08, 4.41151909e-02,
       1.70046794e-08, 5.11503226e-02, 6.82590399e-02, 3.02728712e-02,
       3.79430055e-03, 9.95058777e-02, 1.48755617e-02, 1.10849163e-01,
       1.08087391e-01, 9.45176307e-02, 6.51331697e-02, 2.31402810e-03])

**Creación del modelo de referencia (Cartera Equiponderada)**

Para evaluar el desempeño del modelo de **diversificación máxima**, lo comparamos con una cartera **equiponderada**.

Una cartera **equiponderada** asigna el mismo peso a todos los activos sin considerar volatilidad, correlación ni riesgo. Si el portafolio consta de 20 activos, cada activo tendrá un peso de ***1/20 = 5%***.

Este enfoque es un **benchmark neutral**, ya que no aplica ninguna estrategia de optimización avanzada.



In [ ]:
# Creamos el modelo de referencia (equiponderado)
bench = EqualWeighted()

# Entrenamos el modelo de referencia con el conjunto de entrenamiento
bench.fit(X_train)

# Obtenemos los pesos de los activos en la cartera equiponderada
bench.weights_

array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])

## **Análisis de la Diversificación**
El coeficiente de diversificación es una métrica clave que mide cuán distribuido está el riesgo dentro de una cartera. En esta sección, evaluamos este coeficiente para los dos modelos que construimos:

- **Modelo de Diversificación Máxima**: Diseñado para maximizar la diversificación, debería mostrar el coeficiente más alto.
- **Modelo Equiponderado (Benchmark)**: No optimiza la diversificación, por lo que se espera que su coeficiente sea menor.

Entre mayor sea este ratio, más diversificada está la cartera, lo que puede reducir el riesgo sin sacrificar rendimiento.


📌 **Comparación del coeficiente de diversificación en el conjunto de entrenamiento**

Ejecutamos el modelo en los datos de entrenamiento y comparamos la diversificación de ambas estrategias.

In [ ]:
# Generamos las carteras optimizadas para el conjunto de entrenamiento
ptf_model_train = model.predict(X_train)  # Predicción con el modelo de diversificación máxima
ptf_bench_train = bench.predict(X_train)  # Predicción con el modelo equiponderado

# Mostramos los coeficientes de diversificación
print("🔎 Diversification Ratio:")
print(f"    📊 Maximum Diversification model: {ptf_model_train.diversification:0.2f}")
print(f"    📊 Equal Weighted model: {ptf_bench_train.diversification:0.2f}")

🔎 Diversification Ratio:
    📊 Maximum Diversification model: 1.92
    📊 Equal Weighted model: 1.82


## **Predicción del Desempeño en el Conjunto de Pruebas**

Después de ajustar nuestros modelos con los datos de entrenamiento, ahora evaluaremos su desempeño en un conjunto de pruebas.



📌 **¿Por qué es importante la predicción?**

La predicción en el conjunto de pruebas nos permite validar la robustez del modelo de diversificación máxima y compararlo con el benchmark. Si la estrategia mantiene un buen desempeño en datos nunca vistos, significa que la optimización fue efectiva.

In [ ]:
# Generamos las carteras optimizadas en el conjunto de pruebas
ptf_model_test = model.predict(X_test)  # Predicción con el modelo de diversificación máxima
ptf_bench_test = bench.predict(X_test)  # Predicción con el modelo equiponderado

## **Análisis del Desempeño de las Carteras**
Después de realizar la predicción en el conjunto de pruebas, es fundamental analizar el comportamiento de las carteras optimizadas.
Para ello, utilizaremos la clase `Population` de `skfolio`, que nos permitirá comparar fácilmente las estrategias.



**Carga de las carteras en un objeto `Population`**

`Population` es una clase contenedora que permite manejar múltiples carteras de inversión y facilita su comparación. En este caso, contiene las carteras optimizadas con **Máxima Diversificación** y **Equiponderada**.

In [ ]:
# Creamos un objeto Population con las carteras predichas
population = Population([ptf_model_test, ptf_bench_test])

**Análisis de la Composición de las Carteras**

Podemos visualizar cómo están distribuidas las inversiones dentro de cada cartera:



In [ ]:
# Graficamos la composición de cada cartera
fig = population.plot_composition()
show(fig)

**Evaluación del Rendimiento Acumulado**



In [ ]:
# Graficamos los rendimientos acumulados de cada estrategia en el tiempo
population.plot_cumulative_returns()

**Resumen Completo del Desempeño**



In [ ]:
# Mostramos un resumen detallado de las métricas de cada cartera
population.summary()

,MaximumDiversification,EqualWeighted
Mean,0.072%,0.069%
Annualized Mean,18.17%,17.30%
Variance,0.012%,0.012%
Annualized Variance,3.14%,2.94%
Semi-Variance,0.0062%,0.0060%
Annualized Semi-Variance,1.55%,1.52%
Standard Deviation,1.12%,1.08%
Annualized Standard Deviation,17.71%,17.15%
Semi-Deviation,0.79%,0.78%
Annualized Semi-Deviation,12.46%,12.32%


## **Conclusión**
En este cuaderno, hemos explorado la estrategia de **Diversificación Máxima**, comparándola con una cartera **Equiponderada** para evaluar su efectividad en la optimización del riesgo y la rentabilidad. A través del análisis de la composición, rendimiento y métricas clave, llegamos a las siguientes conclusiones:

- La cartera de **máxima diversificación** logra una mejor distribución del riesgo, reduciendo la concentración en pocos activos.
- El **ratio de diversificación** es significativamente mayor en comparación con la estrategia equiponderada, lo que indica una mejor dispersión del riesgo.
- El **rendimiento acumulado** varía según las condiciones del mercado, por lo que su efectividad debe analizarse en diferentes entornos económicos.